In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import StepLR
import psutil
import os

In [2]:
data_dir = '../input/age-predict/train'
def load_split_train_test(datadir, valid_size = .1):
    train_transforms = transforms.Compose([transforms.Resize((224,224)),
                                           transforms.RandomCrop([196, 196]),
                                           transforms.RandomHorizontalFlip(),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
                                          ])
    test_transforms = transforms.Compose([transforms.Resize((224,224)),
                                          transforms.ToTensor(),
                                          transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
                                         ])
    train_data = datasets.ImageFolder(datadir, transform=train_transforms)
    test_data = datasets.ImageFolder(datadir, transform=test_transforms)    
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=64)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=64)
    return trainloader, testloader
trainloader, testloader = load_split_train_test(data_dir, .2)
print(trainloader.dataset.classes)

['001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100']


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
model = torch.load('../input/recursive-training/aerialmodel.pth')
#model = models.resnet18(pretrained=True)
print(device)

cuda


In [4]:
for param in model.parameters():
    param.requires_grad = True
    
#model.fc = nn.Sequential(nn.Linear(512, 128),
#                         nn.ReLU(),
#                         nn.Dropout(0.2),
#                         nn.Linear(128, 1))
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0000005)
scheduler = StepLR(optimizer, step_size=1, gamma=0.1)
model.to(device);

In [5]:
def forward_model_train(inputs, labels):
    inputs, labels = inputs.to(device), labels.to(device).type(torch.float32)
    #sceduler
    optimizer.zero_grad()
    #scheduler.step()
    logps = torch.flatten(model.forward(inputs))
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()
    return loss.item()
def forward_model_val(inputs, labels):
    inputs, labels = inputs.to(device), labels.to(device).type(torch.float32)
    logps = torch.flatten(model.forward(inputs))
    batch_loss = criterion(logps, labels)
    return batch_loss.item()

In [6]:
epochs = 1
steps = 0
running_loss = 0
print_every = 50
train_losses, test_losses = [], []
for epoch in range(epochs):
    print(scheduler.get_lr())
    scheduler.step()
    print(scheduler.get_lr())
    for inputs, labels in trainloader:
        steps += 1
        running_loss += forward_model_train(inputs, labels)
        #torch.cuda.ipc_collect()
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    test_loss += forward_model_val(inputs, labels)
                    #torch.cuda.ipc_collect()
                    
            print(f"Epoch {epoch + 1}/{epochs}.. "
                  f"Train loss: {running_loss / print_every:.3f}.. "
                  f"Test loss: {test_loss / len(testloader):.3f}.. ")
            running_loss = 0
            model.train()
torch.save(model, 'aerialmodel.pth')

[5e-07]
[5e-09]


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 1/1.. Train loss: 58.050.. Test loss: 54.188.. 
Epoch 1/1.. Train loss: 56.663.. Test loss: 55.998.. 
Epoch 1/1.. Train loss: 55.349.. Test loss: 54.914.. 
Epoch 1/1.. Train loss: 55.764.. Test loss: 54.958.. 
Epoch 1/1.. Train loss: 60.649.. Test loss: 55.165.. 
Epoch 1/1.. Train loss: 57.000.. Test loss: 54.441.. 
Epoch 1/1.. Train loss: 56.889.. Test loss: 56.074.. 
Epoch 1/1.. Train loss: 57.231.. Test loss: 55.204.. 
Epoch 1/1.. Train loss: 57.427.. Test loss: 54.401.. 
Epoch 1/1.. Train loss: 57.763.. Test loss: 54.682.. 
Epoch 1/1.. Train loss: 58.595.. Test loss: 55.123.. 
Epoch 1/1.. Train loss: 58.904.. Test loss: 54.605.. 
Epoch 1/1.. Train loss: 56.146.. Test loss: 55.701.. 
Epoch 1/1.. Train loss: 59.642.. Test loss: 54.561.. 
Epoch 1/1.. Train loss: 59.368.. Test loss: 55.695.. 
Epoch 1/1.. Train loss: 56.983.. Test loss: 54.565.. 
Epoch 1/1.. Train loss: 58.067.. Test loss: 56.066.. 
Epoch 1/1.. Train loss: 57.926.. Test loss: 55.781.. 
Epoch 1/1.. Train loss: 61.5